In [1]:
%load_ext tensorboard

In [2]:
%tensorboard --logdir /workspace/tf_native/results/event_files

In [3]:
import os 
import pandas as pd 
detail=pd.read_csv('./results/simple_tf1_mlp_detailed.csv')
detail.head()


,,,,,,,,,,,,,,,,,,,,,,All Iterations
Op Name,Op Type,No. Calls,TC Eligible,Using TC,Total CPU Time (ns),Avg. CPU Time (ns),Min CPU Time (ns),Max CPU Time (ns),Total GPU Time (ns),Avg. GPU Time (ns),Min GPU Time (ns),Max GPU Time (ns),Total CPU Overhead (ns),Avg. CPU Overhead (ns),Min CPU Overhead (ns),Max CPU Overhead (ns),Total GPU Idle (ns),Avg. GPU Idle (ns),Min GPU Idle (ns),Max GPU Idle (ns),Data Type,Input Shapes
Add,Add,1010,no,no,27264798,26994.8,17966,202025,2750116,2722.9,2495,14784,12149221,12028.9,7746,131785,0,0.0,0,0,float32,"MatMul: <100, 500>, Variable_2/read: <500>"
GradientDescent/GradientDescent/-apply,NoOp,1010,no,no,3471225,3436.9,2625,21267,0,0.0,0,0,3471225,3436.9,2625,21267,0,0.0,0,0,NaN,NaN
GradientDescent/learning_rate,Const,1011,no,no,2724431,2694.8,1394,22408,0,0.0,0,0,2724431,2694.8,1394,22408,0,0.0,0,0,NaN,NaN
GradientDescent/update_Variable/ApplyGradientDescent,ApplyGradientDescent,1010,no,no,11297316,11185.5,9684,28974,2369156,2345.7,2303,2848,4156918,4115.8,3526,21333,0,0.0,0,0,float32,"Variable: <10, 500>, gradients/MatMul_grad/Mat..."


## ---------------- must convert the below to a **xxx.py** file for DLprof to run -----------------

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.head())
# find out how many records we have
len(dataset)
#find out how many columns we have
len(dataset.columns)
X = dataset.iloc[:, 3:13].values # python index does not include the last column=13

type(X)
y = dataset.iloc[:, 13].values # but do it this way, it will only take column 13
type(y)

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

numpy.ndarray

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

X.shape


(10000, 10)

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(type(X_train))
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

<class 'numpy.ndarray'>
(8000, 10)
(8000,)
(2000, 10)
(2000,)


In [5]:
# Feature Scaling - in general, we need to always do feature scaling for neural network
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
records, input_num =X_train.shape
input_num

10

In [6]:
num_labels = 2
# make the training dataset into 200000x 28*28 --> 20000 , 784
def reformat(labels):
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return labels
y_train = reformat(y_train)
y_test = reformat(y_test)

print('Training set', X_train.shape, y_train.shape)
print('testing set', X_test.shape, y_test.shape)

Training set (8000, 10) (8000, 2)
testing set (2000, 10) (2000, 2)


In [7]:
X_test.astype(np.float32)
print(X_test.shape)
type(X_test)

(2000, 10)


numpy.ndarray

In [9]:
# improve your nn by inserting one hidden layer
num_steps = 1000 # hyper-parameter
batch_size = 100 # another hyper-parameter
train_subset = 8000 # in case your memory capacity is low

num_labels=2

# neural network with 1 hidden layer 
n_hidden_1 = 500 #  1st layer number of features, yet another hyper-parameter
n_input = 10    #  data input (img shape: 28*28) since your input matrix shape is (train_subsetX28*28)=10000*784
n_classes = 2    #  total classes 2:churn/not churn 


# Store layers weight & bias using python dictionary-like syntax
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes]))
} # you can access hidden layer1=h1's matrix via weights['h1'] as well as output from activation function via weight['out']

biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'out': tf.Variable(tf.zeros([n_classes]))
}
#print("shape of weight matrix h1: ", weights['h1'])
#print("shape of weight matrix out : ", weights['out'])
#print("shape of bias matrix b1 :", biases['b1'])
#print("shape of bias matrix out", biases['out'])
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input]) # None means you can input dynamic amount of inputs images
y = tf.placeholder(tf.float32, [None, n_classes]) 
#print("x is of shape :", x)
#print("y is of shape :", y)


tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, 10))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.placeholder(tf.float32,
                                    shape=(None, 10))

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #print("shape of layer_1 matrix :" ,layer_1)
    layer_1 = tf.nn.relu(layer_1) 
          # activation function=rectifier , relu(features, name=none) same shape of the layer_1 matrix
    #print("shape after activation function of layer 1: ", layer_1)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    print("shape of out_layer matrix out of hidden layer --> out_layer :" , out_layer)
    return out_layer


In [10]:
# Construct model 
pred = multilayer_perceptron(x, weights, biases)
valid_pred = multilayer_perceptron(tf_valid_dataset, weights, biases)

 
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

shape of out_layer matrix out of hidden layer --> out_layer : Tensor("add_1:0", shape=(?, 2), dtype=float32)
shape of out_layer matrix out of hidden layer --> out_layer : Tensor("add_3:0", shape=(?, 2), dtype=float32)


In [11]:
# Accuracy
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
      / predictions.shape[0])

In [12]:
with tf.Session() as sess:
    sess.run(init)
    print("Initialized tensorflow session")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = X_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = sess.run([optimizer, loss, pred], feed_dict=feed_dict)
        if (step % 100 == 0):
            test_dict={x:X_test, y: y_test}
            _, l, testpred = sess.run([optimizer, loss, pred], feed_dict=test_dict)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("test data accuracy: %.1f%%" % accuracy(testpred, y_test))
            
      

Initialized tensorflow session
Minibatch loss at step 0: 88.495857
Minibatch accuracy: 32.0%
test data accuracy: 79.8%
Minibatch loss at step 100: 2.997778
Minibatch accuracy: 83.0%
test data accuracy: 83.0%
Minibatch loss at step 200: 1.330644
Minibatch accuracy: 87.0%
test data accuracy: 84.9%
Minibatch loss at step 300: 1.977455
Minibatch accuracy: 72.0%
test data accuracy: 83.8%
Minibatch loss at step 400: 1.391028
Minibatch accuracy: 79.0%
test data accuracy: 84.5%
Minibatch loss at step 500: 0.791865
Minibatch accuracy: 78.0%
test data accuracy: 84.1%
Minibatch loss at step 600: 0.697800
Minibatch accuracy: 83.0%
test data accuracy: 75.2%
Minibatch loss at step 700: 0.696401
Minibatch accuracy: 78.0%
test data accuracy: 83.8%
Minibatch loss at step 800: 0.506471
Minibatch accuracy: 85.0%
test data accuracy: 85.0%
Minibatch loss at step 900: 0.541145
Minibatch accuracy: 79.0%
test data accuracy: 85.5%
